<a href="https://colab.research.google.com/github/YuxingW/deep_learning/blob/main/tensorflow_kmean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##This notebook is to use tensorflow KMean algorithm to make clusters from a dataset
###The data set is statistic of number of clients based the ISO installation time on them
###1h -- Number of clients with ISO installation time in 0h~1h
###2h -- Number of clients with ISO installation time in 1h~2h
###3h -- Number of clients with ISO installation time in 2h~3h

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import files
import io

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['dataset.csv'])) #.to_numpy()
df.head()

Saving dataset.csv to dataset (4).csv


,1h,2h,3h,label
0,58,140,25,0
1,49,144,33,0
2,60,144,26,0
3,60,124,35,0
4,58,132,31,0


In [16]:
num_points = 33
dimensions = 3
num_clusters = 3
#points = np.random.uniform(0, 1000, [num_points, dimensions])
points = df.drop(['label'], axis=1).values #returns a numpy array

def input_fn():
  return tf.compat.v1.train.limit_epochs(
      tf.convert_to_tensor(points, dtype=tf.float32), num_epochs=1)

kmeans = tf.compat.v1.estimator.experimental.KMeans(
    num_clusters=num_clusters, use_mini_batch=False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpg1y0o46h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train

In [17]:
num_iterations = 10
previous_centers = None
for _ in range(num_iterations):
  kmeans.train(input_fn)
  cluster_centers = kmeans.cluster_centers()
  if previous_centers is not None:
    print('delta:', cluster_centers - previous_centers)
  previous_centers = cluster_centers
  print('score:', kmeans.score(input_fn))
print('cluster centers:', cluster_centers)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpg1y0o46h/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 1...
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpg1y0o46h/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1...
INFO:tensorflow:Loss for final step: None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-02-07T06:21:13Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpg1y0o46h/model.ckpt-1
INFO:tensorflow:Running local_

## Map the input points to their clusters

In [19]:
cluster_indices = list(kmeans.predict_cluster_index(input_fn))
print(cluster_indices)
for i, point in enumerate(points):
  cluster_index = cluster_indices[i]
  center = cluster_centers[cluster_index]
  print('point:', point, 'is in cluster', cluster_index, 'centered at', center)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpg1y0o46h/model.ckpt-19
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
point: [ 58 140  25] is in cluster 1 centered at [ 55.142853 139.14284   30.190474]
point: [ 49 144  33] is in cluster 1 centered at [ 55.142853 139.14284   30.190474]
point: [ 60 144  26] is in cluster 1 centered at [ 55.142853 139.14284   30.190474]
point: [ 60 124  35] is in cluster 1 centered at [ 55.142853 139.14284   30.190474]
point: [ 58 132  31] is in cluster 1 centered at [ 55.142853 139.14284   30.190474]
point: [ 23 163  43] is in cluster 2 centered at [ 11.299999 167.79999   47.899994]
point: [ 18 175  37] is in cluster 2 centered at [ 11.299999 167.79999   47.899994]
point: [  7  98 124] is in cluster 0 centered 

Refference:
https://www.tensorflow.org/api_docs/python/tf/compat/v1/estimator/experimental/KMeans